In [1]:
from pyspark.sql import SparkSession
# Inicializa uma sessão do Spark com as configurações do Iceberg

spark = SparkSession.builder \
  .appName("IcebergLocalDevelopment") \
  .config("spark.driver.host", "localhost") \
  .config('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1') \
  .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
  .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
  .config("spark.sql.catalog.local.type", "hadoop") \
  .config("spark.sql.catalog.local.warehouse", "spark-warehouse/iceberg") \
  .config("spark.driver.bindAddress", "localhost") \
  .getOrCreate()
